# 0. Импортируем, загружаем, подготавливаем

In [0]:
!pip install textdistance

In [0]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import textdistance
from tqdm import tqdm

In [0]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [0]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [0]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf-8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [0]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

## Удаляем неточности в align_words

Align_words может не всегда работать точно. Так, на примере ниже показано, что правильные и неправильные варианты могут сьезжать, хотя при этом их кол-во будет соответствовать.

In [7]:
align_words(bad[9], true[9])

[('вобщем', 'в'),
 ('как', 'общем'),
 ('вы', 'как'),
 ('знаете', 'вы'),
 ('из', 'знаете'),
 ('моего', 'из'),
 ('не', 'моего'),
 ('давнего', 'недавнего'),
 ('поста', 'поста'),
 ('я', 'я'),
 ('жаловался', 'жаловался'),
 ('на', 'на'),
 ('пропажу', 'пропажу'),
 ('писем', 'писем'),
 ('с', 'с'),
 ('моего', 'моего'),
 ('ящека', 'ящика'),
 ('на', 'на'),
 ('почте.ру', 'почте.ру')]

Для начала найдем похожие случаи с помощью простых предположений, проверим каждый из случаев, и если в нем будет все съезжать, то удалим его.

In [8]:
#Попробуем найти похожие примеры, если они есть.
testik = []
bad_to_del = []
true_to_del = []
for i in range(len(bad)):
    if ('вобщем' in bad[i] or 'вощем' in bad[i]) and  'в общем' in true[i]: #случай когда 'вобщем' делится на 'в' и 'общем'
        testik.append(i)
        bad_to_del.append(bad[i])
        true_to_del.append(true[i])
    elif ('как то' in bad[i]) and 'как-то' in true[i]:
        testik.append(i)
        bad_to_del.append(bad[i])
        true_to_del.append(true[i])
testik

[9, 276, 468, 722]

In [9]:
align_words(bad[276], true[276])

[('вобщем', 'в'),
 ('сниццо', 'общем'),
 ('мне', 'снится'),
 ('что', 'мне'),
 ('я', 'что'),
 ('нашел', 'я'),
 ('таки', 'нашел-таки'),
 ('артефакт', 'артефакт'),
 ('позволяющий', 'позволяющий'),
 ('видеть', 'видеть'),
 ('что', 'что'),
 ('там', 'там'),
 ('у', 'у'),
 ('людей', 'людей'),
 ('внутри', 'внутри')]

In [10]:
align_words(bad[468], true[468])

[('ура', 'ура'),
 ('я', 'я'),
 ('сделала', 'сделала'),
 ('себе', 'себе'),
 ('выходной', 'выходной'),
 ('правда', 'правда'),
 ('провела', 'провела'),
 ('его', 'его'),
 ('как', 'как-то'),
 ('то', 'странно'),
 ('странно', 'на'),
 ('на', 'улице'),
 ('улице', 'супер'),
 ('супер', 'просто'),
 ('просто', 'а'),
 ('а', 'я'),
 ('я', 'целый'),
 ('целый', 'день'),
 ('день', 'проспала'),
 ('проспала', 'потом'),
 ('потом', 'уборкой'),
 ('уборкой', 'занималась'),
 ('занималась', 'ну'),
 ('ну', 'в'),
 ('вобщем', 'общем'),
 ('как', 'как'),
 ('обычно', 'обычно')]

In [11]:
align_words(bad[722], true[722])

[('что', 'что'),
 ('примечательно', 'примечательно'),
 ('перед', 'перед'),
 ('нами', 'нами'),
 ('выступали', 'выступали'),
 ('мои', 'мои'),
 ('знакомые', 'знакомые'),
 ('с', 'с'),
 ('реп', 'репбазы'),
 ('базы', 'которые'),
 ('которые', 'оказались'),
 ('оказались', 'крайне'),
 ('крайне', 'близкими'),
 ('близкими', 'друзьями'),
 ('друзьями', 'нежданным'),
 ('нежданным', 'в'),
 ('вобщем', 'общем'),
 ('море', 'море'),
 ('позитива', 'позитива'),
 ('и', 'и'),
 ('рокенрола', 'рок-н-ролла')]

In [0]:
#удаляем эти предложения из набора
for i in testik[::-1]:
    true.remove(true[i])
    bad.remove(bad[i])

Удалив ненужное, посмотрим на долю ошибок.

In [0]:
mistakes_list = []
total = 0

for i in range(len(true)):
    
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes_list.append(pair)
        total += 1

In [14]:
print('Доля ошибок - ', len(mistakes_list)/total )

Доля ошибок -  0.1270896273917422


In [15]:
1 - len(mistakes_list)/total

0.8729103726082578

# 1. Совмещаем get_closest_match_vec и Левенштейна

Функция get_closest_vector_levi сначала находит TOPN (по дефолту = 6) вариантов по векторной близости, затем для каждого из кандидатов находит расстояние Левенштейна и вовращает наилучший вариант.

In [0]:
def get_closest_match_vec(text, X, vec, TOPN=6):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [0]:
def get_closest_vector_levi(text, X, vec, metric=textdistance.levenshtein):
    #v = vec.transform([text])
    # similarities = cosine_distances(v, X)
    #topn = similarities.argsort()[0][:TOPN]
    #variants = [id2word[top] for top in topn]
    variants = get_closest_match_vec(text, X, vec, TOPN=6)
    similarities = [metric.normalized_similarity(text, variant) for variant in variants]
    best = variants[similarities.index(max(similarities))]
    return best

In [18]:
%%time
get_closest_vector_levi('вобще',  X, vec)

CPU times: user 55.2 ms, sys: 2.9 ms, total: 58.1 ms
Wall time: 63 ms


'вообще'

In [19]:
%%time
get_closest_vector_levi('опофеоз',  X, vec)

CPU times: user 55.9 ms, sys: 852 µs, total: 56.7 ms
Wall time: 60.1 ms


'апофеоз'

In [20]:
%%time
get_closest_vector_levi('очень',  X, vec)

CPU times: user 52.5 ms, sys: 1.94 ms, total: 54.4 ms
Wall time: 57 ms


'очень'

# 2. Проверяем качество на данных соревнования диалога

In [21]:
#Создадим словарь неверных исправлений
mistakes = {'bad':[], 'our correction':[], 'true':[]}


correct = 0
total = 0
for i in tqdm(range(len(true))):
    word_pairs = align_words(bad[i], true[i])
    for pair in word_pairs:
        predicted = get_closest_vector_levi(pair[0],  X, vec)
        if predicted == pair[1]:
            correct += 1
        else:
          mistakes['bad'].append(pair[0])
          mistakes['our correction']. append(predicted)
          mistakes['true'].append(pair[1])          
        total += 1
    

100%|██████████| 912/912 [08:43<00:00,  1.38it/s]


In [22]:
correct/total

0.8326283987915408

# 3. Смотрим на ошибки

In [0]:
import pandas as pd

In [24]:
df_mistakes = pd.DataFrame(mistakes)
print('Кол-во ошибок: ', len(df_mistakes['bad']))
print('Доля ошибок:', len(df_mistakes) / total)
df_mistakes.head(10)

Кол-во ошибок:  1662
Доля ошибок: 0.1673716012084592


,bad,our correction,true
0,симпатичнейшое,пластичнейшими,симпатичнейшее
1,шпионское,шпионские,шпионское
2,гламурный,лагерный,гламурный
3,бонда,банда,бонда
4,superheadz,heads,superheadz
5,clap,place,clap
6,camera,america,camera
7,полчатся,ополчатся,получатся
8,язычки,язычка,язычки
9,оччччень,чечни,очень


In [26]:
%%time
get_closest_vector_levi('оччччень',  X, vec)

CPU times: user 53.9 ms, sys: 1.98 ms, total: 55.9 ms
Wall time: 57 ms


'чечни'

Многие ошибки связаны с тем, что корректор исправляет уже верные слова на ошибочные. Возможно, это связано с тем, что нашего словаря недостаточно. 

# 4. Исправляем ошибки

## 4.1 Попробуем использовать буквенные нграммы

In [0]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,4))
X = vec.fit_transform(vocab)

In [0]:
def get_closest_match_vec(text, X, vec, TOPN=6):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [0]:
def get_closest_vector_levi(text, X, vec, metric=textdistance.levenshtein):
    #v = vec.transform([text])
    # similarities = cosine_distances(v, X)
    #topn = similarities.argsort()[0][:TOPN]
    #variants = [id2word[top] for top in topn]
    variants = get_closest_match_vec(text, X, vec, TOPN=6)
    similarities = [metric.normalized_similarity(text, variant) for variant in variants]
    best = variants[similarities.index(max(similarities))]
    return best

In [30]:
mistakes2 = {'bad':[], 'our correction':[], 'true':[]}


correct = 0
total = 0
for i in tqdm(range(len(true))):
    word_pairs = align_words(bad[i], true[i])
    for pair in word_pairs:
        predicted = get_closest_vector_levi(pair[0],  X, vec)
        if predicted == pair[1]:
            correct += 1
        else:
          mistakes2['bad'].append(pair[0])
          mistakes2['our correction']. append(predicted)
          mistakes2['true'].append(pair[1])          
        total += 1

100%|██████████| 912/912 [41:01<00:00,  3.47s/it]


In [31]:
df_mistakes2 = pd.DataFrame(mistakes2)
print(len(df_mistakes2['bad']))
print('Доля ошибок:', len(df_mistakes2) / total)
df_mistakes2.head(10)

1527
Доля ошибок: 0.1537764350453172


,bad,our correction,true
0,симпатичнейшое,симпатичный,симпатичнейшее
1,шпионское,шпионской,шпионское
2,гламурный,гламур,гламурный
3,бонда,бонди,бонда
4,superheadz,superjet,superheadz
5,clap,la,clap
6,camera,tamer,camera
7,полчатся,ополчатся,получатся
8,язычки,язычка,язычки
9,милые,милы,милые


In [32]:
correct/total

0.8462235649546828

Как мы видим, кол-во ошибок уменьшилось, но незначительно. 

## 4.2 Добавим pymorphy

Используем его так: когда подбираем варианты, смотрим на то, какой они части речи. Если совпадают с исходной, оставляем, если нет - выкидываем, несмотря на вероятность. Однако, если не остается ни одного варианта исходной части речи, то работаем с дефолтными вариантами.

In [0]:
!pip install pymorphy2
import pymorphy2 as py
morph = py.MorphAnalyzer()

In [0]:
def get_closest_vector_levi(text, X, vec, metric=textdistance.levenshtein):
    #v = vec.transform([text])
    # similarities = cosine_distances(v, X)
    #topn = similarities.argsort()[0][:TOPN]
    #variants = [id2word[top] for top in topn]
    variants = get_closest_match_vec(text, X, vec, TOPN=6)
    similarities = [textdistance.levenshtein.normalized_similarity(text, variant) for variant in variants]
    #print(variants, similarities)
    variants, similarities = pos_similar(text, variants, similarities)
    #print(variants, similarities)
    best = variants[similarities.index(max(similarities))]
    return best

In [0]:
def pos_similar(mistake, variants, similarities):
  word = morph.parse(mistake)[0]
  t = word.tag.POS
  new_variants = []
  new_similarities = []
  for variant in variants:
    if morph.parse(variant)[0].tag.POS == t:
      new_variants.append(variant)
      new_similarities.append(similarities[variants.index(variant)])
    else:
      pass
  if len(new_variants) > 0:
    return new_variants, new_similarities
  else:
    return variants, similarities

Обработаем так только ошибки. 

In [36]:
mistakes_after_pos = {'bad':[], 'our correction':[], 'true':[]}
n_mistakes = len(df_mistakes2)
y_add_pred = []
for i in tqdm(range(len(mistakes2['bad']))):
  mistake = mistakes2['bad'][i]
  pred = get_closest_vector_levi(mistake,  X, vec)
  if pred != mistakes2['true'][i]:
    mistakes_after_pos['bad'].append(mistakes2['bad'][i])
    mistakes_after_pos['our correction'].append(pred)
    mistakes_after_pos['true'].append(mistakes2['true'][i])
  else:
    n_mistakes -= 1
  

100%|██████████| 1527/1527 [06:30<00:00,  3.88it/s]


In [37]:
df_mistakes3 = pd.DataFrame(mistakes_after_pos)
print('Кол-во ошибок: ', len(df_mistakes3['bad']))
print('Доля ошибок:', len(df_mistakes3) / total)
df_mistakes3.head(10)

Кол-во ошибок:  1518
Доля ошибок: 0.1528700906344411


,bad,our correction,true
0,симпатичнейшое,симпатией,симпатичнейшее
1,шпионское,шпионской,шпионское
2,гламурный,пасмурный,гламурный
3,бонда,бонди,бонда
4,superheadz,superjet,superheadz
5,clap,la,clap
6,camera,tamer,camera
7,полчатся,ополчатся,получатся
8,язычки,язычка,язычки
9,милые,милым,милые


In [38]:
1 - n_mistakes/total

0.8471299093655589

Как видно, совсем незначительное число ошибок удалось исправить таким способом.